In [1]:
import pandas as pd

rule_result_df = pd.read_excel("../result/rule_matching_result_ver3.xlsx")
bd_df = pd.read_excel("../data/BD_data_all.xlsx")

In [2]:
import os
import re
import pandas as pd

# --- 1. 텍스트 전처리 함수 ---
def tokenize_text(text):
    """
    1) 소문자 변환
    2) 괄호 등 모든 특수문자를 공백으로 치환
    3) 공백 기준으로 토큰 분할 후 리스트 반환
    """
    if pd.isna(text):
        return []
    text = text.lower()
    # 특수문자(영문자, 숫자, 한글, 공백 제외) 모두 공백으로 대체
    text = re.sub(r'[^a-z0-9가-힣\s]', ' ', text)
    text = ' '.join(text.split())
    return text.split()

def preprocess_data(df):
    """
    df에 포함된 텍스트 컬럼에 대해 토큰화 열을 추가합니다.
    EBD: '기관명', '건축물명', '주소'
    BD: 'BLD_NM', 'DONG_NM'
    이미 존재하는 컬럼이 있을 경우에만 처리합니다.
    """
    df = df.copy()
    text_cols = ['기관명', '건축물명', '주소', 'BLD_NM', 'DONG_NM']
    for col in text_cols:
        if col in df.columns:
            token_col = col + '_tokens'
            df[token_col] = df[col].apply(tokenize_text)
    return df

# --- 2. 텍스트 점수 계산 함수 ---
def compare_tokens(ebd_tokens, bd_tokens_1, bd_tokens_2):
    """
    EBD의 토큰 리스트와, BD의 두 토큰 리스트(예: BLD_NM_tokens, DONG_NM_tokens)를 합집합으로 만들고,
    교집합이 1개 이상이면 1점을, 없으면 0점을 반환.
    """
    if not ebd_tokens:
        return 0.0
    bd_union = set(bd_tokens_1).union(set(bd_tokens_2))
    if set(ebd_tokens).intersection(bd_union):
        return 1.0
    else:
        return 0.0

def calculate_text_score(ebd_row, bd_row):
    """
    EBD의 3가지 변수와 BD의 2가지 변수의 토큰을 비교하여 텍스트 점수를 계산합니다.
      - office_score: EBD '기관명_tokens' vs BD ('BLD_NM_tokens' + 'DONG_NM_tokens')
      - bld_score:    EBD '건축물명_tokens' vs BD ('BLD_NM_tokens' + 'DONG_NM_tokens')
      - juso_score:   EBD '주소_tokens' vs BD ('BLD_NM_tokens' + 'DONG_NM_tokens')
    최종 text_score는 각 점수의 합입니다.
    """
    office_score = compare_tokens(
        ebd_row.get('기관명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    bld_score = compare_tokens(
        ebd_row.get('건축물명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    juso_score = compare_tokens(
        ebd_row.get('주소_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    return office_score, bld_score, juso_score, office_score + bld_score + juso_score



# --- 4. 데이터 전처리 ---
# EBD 데이터 (rule_result_df)와 BD 데이터 모두에 대해 토큰화 수행
ebd_processed = preprocess_data(rule_result_df)
bd_processed = preprocess_data(bd_df)


=== Sample EBD 건물 ===
기관명: 감사원
건축물명: 본관
주소: 서울특별시 종로구 25-23
토큰(기관명): ['감사원']
토큰(건축물명): ['본관']
토큰(주소): ['서울특별시', '종로구', '25', '23']
RECAP_PK: 11110-900
------------------------
해당 RECAP의 BD 후보 개수: 5

BD 후보 인덱스 286:
BLD_NM: 본관
DONG_NM: nan
Office score: 0.0
Bld score: 1.0
Juso score: 0.0
Total text score: 1.0
토큰(블록): ['본관'] []
------------------------
BD 후보 인덱스 287:
BLD_NM: 제4별관
DONG_NM: nan
Office score: 0.0
Bld score: 0.0
Juso score: 0.0
Total text score: 0.0
토큰(블록): ['제4별관'] []
------------------------
BD 후보 인덱스 288:
BLD_NM: 제1별관
DONG_NM: nan
Office score: 0.0
Bld score: 0.0
Juso score: 0.0
Total text score: 0.0
토큰(블록): ['제1별관'] []
------------------------
BD 후보 인덱스 289:
BLD_NM: (제5호)
DONG_NM: nan
Office score: 0.0
Bld score: 0.0
Juso score: 0.0
Total text score: 0.0
토큰(블록): ['제5호'] []
------------------------
BD 후보 인덱스 290:
BLD_NM: (제4호)
DONG_NM: nan
Office score: 0.0
Bld score: 0.0
Juso score: 0.0
Total text score: 0.0
토큰(블록): ['제4호'] []
------------------------


In [6]:

# --- 5. RECAP 기반 후보 탐색 및 텍스트 점수 실험 ---
# 예를 들어, 첫 번째 EBD 건을 선택하여 해당 RECAP_PK에 속한 BD 후보들을 가져와 비교
sample_ebd = ebd_processed.iloc[2357]
recap = sample_ebd['RECAP_PK']
print("=== Sample EBD 건물 ===")
print("기관명:", sample_ebd['기관명'])
print("건축물명:", sample_ebd['건축물명'])
print("주소:", sample_ebd['주소'])
print("토큰(기관명):", sample_ebd.get('기관명_tokens', []))
print("토큰(건축물명):", sample_ebd.get('건축물명_tokens', []))
print("토큰(주소):", sample_ebd.get('주소_tokens', []))
print("RECAP_PK:", recap)
print("------------------------")

# 해당 RECAP_PK에 해당하는 BD 후보들 선택
bd_candidates = bd_processed[bd_processed['RECAP_PK'] == recap]
print("해당 RECAP의 BD 후보 개수:", len(bd_candidates))
print()

# 각 BD 후보에 대해 텍스트 점수를 계산하고 출력
for idx, bd_candidate in bd_candidates.iterrows():
    office, bld, juso, total = calculate_text_score(sample_ebd, bd_candidate)
    print(f"BD 후보 인덱스 {idx}:")
    print("BLD_NM:", bd_candidate.get('BLD_NM', ''))
    print("DONG_NM:", bd_candidate.get('DONG_NM', ''))
    print("Office score:", office)
    print("Bld score:", bld)
    print("Juso score:", juso)
    print("Total text score:", total)
    print("토큰(블록):", bd_candidate.get('BLD_NM_tokens', []), bd_candidate.get('DONG_NM_tokens', []))
    print("------------------------")


=== Sample EBD 건물 ===
기관명: 서울대학교
건축물명: 관악 10-1동[사범대교육정보관]
주소: 서울특별시 관악구 관악로 1(신림동 56-1번지, 관악 10-1동[사범대교육정보관] ) 
토큰(기관명): ['서울대학교']
토큰(건축물명): ['관악', '10', '1동', '사범대교육정보관']
토큰(주소): ['서울특별시', '관악구', '관악로', '1', '신림동', '56', '1번지', '관악', '10', '1동', '사범대교육정보관']
RECAP_PK: 11620-100197744
------------------------
해당 RECAP의 BD 후보 개수: 152

BD 후보 인덱스 1360:
BLD_NM: 관악 131동[신소재공동연구소]
DONG_NM: nan
Office score: 0.0
Bld score: 1.0
Juso score: 1.0
Total text score: 2.0
토큰(블록): ['관악', '131동', '신소재공동연구소'] []
------------------------
BD 후보 인덱스 1361:
BLD_NM: 관악 104동[반도체공동연구소]
DONG_NM: nan
Office score: 0.0
Bld score: 1.0
Juso score: 1.0
Total text score: 2.0
토큰(블록): ['관악', '104동', '반도체공동연구소'] []
------------------------
BD 후보 인덱스 1362:
BLD_NM: 관악 103동[규장각한국학연구원]
DONG_NM: nan
Office score: 0.0
Bld score: 1.0
Juso score: 1.0
Total text score: 2.0
토큰(블록): ['관악', '103동', '규장각한국학연구원'] []
------------------------
BD 후보 인덱스 1363:
BLD_NM: 관악 12동[사범교육협력센터]
DONG_NM: nan
Office score: 0.0
Bld score: 1.0
Juso scor

In [13]:
import os
import re
import pandas as pd

# --- 전처리 함수: 텍스트 토큰화 ---
def tokenize_text(text):
    """
    소문자 변환, 특수문자(괄호 등)를 모두 공백으로 치환 후, 공백 기준으로 토큰 분할.
    """
    if pd.isna(text):
        return []
    text = text.lower()
    # 영문, 숫자, 한글, 공백 외의 모든 문자 -> 공백
    text = re.sub(r'[^a-z0-9가-힣\s]', ' ', text)
    text = ' '.join(text.split())
    return text.split()

def preprocess_data(df):
    """
    DataFrame 내에 있는 텍스트 컬럼에 대해 토큰화 열을 추가합니다.
    EBD의 경우: '기관명', '건축물명', '주소'
    BD의 경우: 'BLD_NM', 'DONG_NM'
    """
    df = df.copy()
    # 처리할 컬럼 목록
    cols_to_tokenize = ['기관명', '건축물명', '주소', 'BLD_NM', 'DONG_NM']
    for col in cols_to_tokenize:
        if col in df.columns:
            df[col + '_tokens'] = df[col].apply(tokenize_text)
    return df

# --- 텍스트 점수 계산 함수 ---
def compare_tokens(ebd_tokens, bd_tokens_1, bd_tokens_2):
    """
    EBD의 토큰 리스트와 BD의 두 토큰 리스트(합집합) 사이에 
    공통 토큰이 있으면 1점, 없으면 0점 반환.
    """
    if not ebd_tokens:
        return 0.0
    bd_union = set(bd_tokens_1).union(set(bd_tokens_2))
    return 1.0 if set(ebd_tokens).intersection(bd_union) else 0.0

def calculate_text_score(ebd_row, bd_row):
    """
    EBD의 기관명, 건축물명, 주소 토큰과 BD의 BLD_NM, DONG_NM 토큰을 비교하여
    각각 office_score, bld_score, juso_score를 계산하고, 총합(total_text_score)을 반환.
    """
    office_score = compare_tokens(
        ebd_row.get('기관명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    bld_score = compare_tokens(
        ebd_row.get('건축물명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    juso_score = compare_tokens(
        ebd_row.get('주소_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    total_text_score = office_score + bld_score + juso_score
    return office_score, bld_score, juso_score, total_text_score

# --- 데이터 로드 ---


# --- 데이터 전처리: 토큰 열 추가 ---
ebd_processed = preprocess_data(rule_result_df)
bd_processed = preprocess_data(bd_df)

# --- 예시 실험: 한 EBD 건물에 대해 BD 후보와 토큰 및 텍스트 점수 비교 ---
# 예를 들어, 첫 번째 EBD 행을 선택
sample_ebd = ebd_processed.iloc[2357]
recap = sample_ebd['RECAP_PK']
print("=== Sample EBD 건물 ===")
print("기관명:", sample_ebd['기관명'])
print("건축물명:", sample_ebd['건축물명'])
print("주소:", sample_ebd['주소'])
print("토큰(기관명):", sample_ebd.get('기관명_tokens', []))
print("토큰(건축물명):", sample_ebd.get('건축물명_tokens', []))
print("토큰(주소):", sample_ebd.get('주소_tokens', []))
print("RECAP_PK:", recap)
print("------------------------")

# 해당 RECAP_PK에 속한 BD 후보 선택
bd_candidates = bd_processed[bd_processed['RECAP_PK'] == recap]
print("해당 RECAP의 BD 후보 개수:", len(bd_candidates))
print()

# 각 BD 후보에 대해 토큰과 텍스트 점수를 계산하여 DataFrame에 저장
results = []
for idx, bd_candidate in bd_candidates.iterrows():
    office, bld, juso, total_text = calculate_text_score(sample_ebd, bd_candidate)
    result = {
        'BD_idx': idx,
        'BLD_NM': bd_candidate.get('BLD_NM', ''),
        'DONG_NM': bd_candidate.get('DONG_NM', ''),
        '기관명_tokens': sample_ebd.get('기관명_tokens', []),
        '건축물명_tokens': sample_ebd.get('건축물명_tokens', []),
        '주소_tokens': sample_ebd.get('주소_tokens', []),
        'BLD_NM_tokens': bd_candidate.get('BLD_NM_tokens', []),
        'DONG_NM_tokens': bd_candidate.get('DONG_NM_tokens', []),
        'office_score': office,
        'bld_score': bld,
        'juso_score': juso,
        'total_text_score': total_text
    }
    results.append(result)

scores_df = pd.DataFrame(results)
print("BD 후보별 텍스트 점수 및 토큰 결과:")
scores_df

=== Sample EBD 건물 ===
기관명: 서울대학교
건축물명: 관악 10-1동[사범대교육정보관]
주소: 서울특별시 관악구 관악로 1(신림동 56-1번지, 관악 10-1동[사범대교육정보관] ) 
토큰(기관명): ['서울대학교']
토큰(건축물명): ['관악', '10', '1동', '사범대교육정보관']
토큰(주소): ['서울특별시', '관악구', '관악로', '1', '신림동', '56', '1번지', '관악', '10', '1동', '사범대교육정보관']
RECAP_PK: 11620-100197744
------------------------
해당 RECAP의 BD 후보 개수: 152

BD 후보별 텍스트 점수 및 토큰 결과:


,BD_idx,BLD_NM,DONG_NM,기관명_tokens,건축물명_tokens,주소_tokens,BLD_NM_tokens,DONG_NM_tokens,office_score,bld_score,juso_score,total_text_score
0,1360,관악 131동[신소재공동연구소],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 131동, 신소재공동연구소]",[],0.0,1.0,1.0,2.0
1,1361,관악 104동[반도체공동연구소],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 104동, 반도체공동연구소]",[],0.0,1.0,1.0,2.0
2,1362,관악 103동[규장각한국학연구원],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 103동, 규장각한국학연구원]",[],0.0,1.0,1.0,2.0
3,1363,관악 12동[사범교육협력센터],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 12동, 사범교육협력센터]",[],0.0,1.0,1.0,2.0
4,1364,관악 83동[인문사회계멀티미디어강의동],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 83동, 인문사회계멀티미디어강의동]",[],0.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
147,1507,관악 32-1동[해동학술문화관],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 32, 1동, 해동학술문화관]",[],0.0,1.0,1.0,2.0
148,1508,관악 72동[법학도서관],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 72동, 법학도서관]",[],0.0,1.0,1.0,2.0
149,1509,관악 14동[인문관7],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 14동, 인문관7]",[],0.0,1.0,1.0,2.0
150,1510,관악 18동[자연과학관1],NaN,[서울대학교],"[관악, 10, 1동, 사범대교육정보관]","[서울특별시, 관악구, 관악로, 1, 신림동, 56, 1번지, 관악, 10, 1동,...","[관악, 18동, 자연과학관1]",[],0.0,1.0,1.0,2.0


In [14]:
import os
import re
import pandas as pd

# --- 전처리 함수: 텍스트 토큰화 ---
def tokenize_text(text):
    """
    소문자 변환, 특수문자(괄호 등)를 모두 공백으로 치환 후, 공백 기준으로 토큰 분할.
    """
    if pd.isna(text):
        return []
    text = text.lower()
    # 영문, 숫자, 한글, 공백 외의 모든 문자 -> 공백
    text = re.sub(r'[^a-z0-9가-힣\s]', ' ', text)
    text = ' '.join(text.split())
    return text.split()

def preprocess_data(df):
    """
    DataFrame 내에 있는 텍스트 컬럼에 대해 토큰화 열을 추가합니다.
    EBD의 경우: '기관명', '건축물명', '주소'
    BD의 경우: 'BLD_NM', 'DONG_NM'
    """
    df = df.copy()
    cols_to_tokenize = ['기관명', '건축물명', '주소', 'BLD_NM', 'DONG_NM']
    for col in cols_to_tokenize:
        if col in df.columns:
            df[col + '_tokens'] = df[col].apply(tokenize_text)
    return df

# --- 텍스트 점수 계산 함수 ---
def compare_tokens(ebd_tokens, bd_tokens_1, bd_tokens_2):
    """
    EBD의 토큰 리스트와 BD의 두 토큰 리스트(합집합) 사이에 
    공통 토큰이 있으면 1점, 없으면 0점 반환.
    """
    if not ebd_tokens:
        return 0.0
    bd_union = set(bd_tokens_1).union(set(bd_tokens_2))
    return 1.0 if set(ebd_tokens).intersection(bd_union) else 0.0

def calculate_text_score(ebd_row, bd_row):
    """
    EBD의 기관명, 건축물명, 주소 토큰과 BD의 BLD_NM, DONG_NM 토큰을 비교하여
    각각 office_score, bld_score, juso_score를 계산하고, 총합(total_text_score)을 반환.
    """
    office_score = compare_tokens(
        ebd_row.get('기관명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    bld_score = compare_tokens(
        ebd_row.get('건축물명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    juso_score = compare_tokens(
        ebd_row.get('주소_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    total_text_score = office_score + bld_score + juso_score
    return office_score, bld_score, juso_score, total_text_score

# --- 데이터 로드 ---
# (파일 경로와 파일 이름은 실제 환경에 맞게 수정)


# --- 데이터 전처리: 토큰 열 추가 ---
ebd_processed = preprocess_data(rule_result_df)  # EBD: 기관명, 건축물명, 주소 토큰 생성
bd_processed = preprocess_data(bd_df)             # BD: BLD_NM, DONG_NM 토큰 생성

# --- 전체 EBD에 대해 BD 후보 텍스트 점수 계산 ---
# 각 EBD 행의 RECAP_PK에 해당하는 BD 후보들과 비교하여 점수를 계산합니다.
results = []
for ebd_idx, ebd_row in ebd_processed.iterrows():
    recap = ebd_row['RECAP_PK']
    # RECAP_PK가 같은 BD 후보들 선택
    bd_candidates = bd_processed[bd_processed['RECAP_PK'] == recap]
    if bd_candidates.empty:
        continue
    for bd_idx, bd_row in bd_candidates.iterrows():
        office, bld, juso, total_text = calculate_text_score(ebd_row, bd_row)
        results.append({
            'EBD_idx': ebd_idx,
            'BD_idx': bd_idx,
            'RECAP_PK': recap,
            'office_score': office,
            'bld_score': bld,
            'juso_score': juso,
            'total_text_score': total_text
        })

all_scores_df = pd.DataFrame(results)

# --- 전체 점수 분포 집계 ---
print("전체 EBD의 BD 후보 점수 분포 (전체 텍스트 점수):")
print(all_scores_df['total_text_score'].value_counts())

print("\nOffice score 분포:")
print(all_scores_df['office_score'].value_counts())

print("\nBld score 분포:")
print(all_scores_df['bld_score'].value_counts())

print("\nJuso score 분포:")
print(all_scores_df['juso_score'].value_counts())

# 예시: 전체 점수가 3점인 경우 (각 필드 모두 일치한 경우)
three_point_cases = all_scores_df[all_scores_df['total_text_score'] == 3.0]
print("\n총 텍스트 점수 3점인 후보 수:", len(three_point_cases))



전체 EBD의 BD 후보 점수 분포 (전체 텍스트 점수):
total_text_score
0.0    49210
1.0    13413
2.0     6098
3.0      552
Name: count, dtype: int64

Office score 분포:
office_score
0.0    57001
1.0    12272
Name: count, dtype: int64

Bld score 분포:
bld_score
0.0    63212
1.0     6061
Name: count, dtype: int64

Juso score 분포:
juso_score
0.0    60341
1.0     8932
Name: count, dtype: int64

총 텍스트 점수 3점인 후보 수: 552


In [15]:
import os
import re
import pandas as pd
from tqdm import tqdm

# --- 전처리 함수: 텍스트 토큰화 ---
def tokenize_text(text):
    """
    소문자 변환, 특수문자(괄호 등)를 모두 공백으로 치환 후, 
    공백 기준으로 토큰 분할하여 리스트를 반환합니다.
    """
    if pd.isna(text):
        return []
    text = text.lower()
    # 영문, 숫자, 한글, 공백 외의 모든 문자 -> 공백으로 치환
    text = re.sub(r'[^a-z0-9가-힣\s]', ' ', text)
    text = ' '.join(text.split())
    return text.split()

# --- 데이터 전처리 함수 ---
def preprocess_data(df):
    """
    DataFrame 내에 있는 텍스트 컬럼에 대해 토큰화 열을 추가합니다.
    EBD의 경우: '기관명', '건축물명', '주소'
    BD의 경우: 'BLD_NM', 'DONG_NM'
    """
    df = df.copy()
    cols_to_tokenize = ['기관명', '건축물명', '주소', 'BLD_NM', 'DONG_NM']
    for col in cols_to_tokenize:
        if col in df.columns:
            df[col + '_tokens'] = df[col].apply(tokenize_text)
    return df

# --- 텍스트 점수 계산 함수 ---
def compare_tokens(ebd_tokens, bd_tokens_1, bd_tokens_2):
    """
    EBD의 토큰 리스트와 BD의 두 토큰 리스트(합집합) 사이에
    공통 토큰이 있으면 1점, 없으면 0점을 반환합니다.
    """
    if not ebd_tokens:
        return 0.0
    bd_union = set(bd_tokens_1).union(set(bd_tokens_2))
    return 1.0 if set(ebd_tokens).intersection(bd_union) else 0.0

def calculate_text_score(ebd_row, bd_row):
    """
    EBD의 기관명, 건축물명, 주소 토큰과 BD의 BLD_NM, DONG_NM 토큰을 비교하여,
    각각 office_score, bld_score, juso_score를 계산하고,
    총합(total_text_score)을 반환합니다.
    """
    office_score = compare_tokens(
        ebd_row.get('기관명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    bld_score = compare_tokens(
        ebd_row.get('건축물명_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    juso_score = compare_tokens(
        ebd_row.get('주소_tokens', []),
        bd_row.get('BLD_NM_tokens', []),
        bd_row.get('DONG_NM_tokens', [])
    )
    total_text_score = office_score + bld_score + juso_score
    return office_score, bld_score, juso_score, total_text_score

# --- 데이터 로드 ---
def load_data():
    """
    RULE 매칭 결과(즉, EBD 데이터)와 BD_all 데이터를 엑셀 파일에서 로드합니다.
    """
    rule_result_df = pd.read_excel("../result/rule_matching_result_ver3.xlsx")
    bd_df = pd.read_excel("../data/BD_data_all.xlsx")
    return rule_result_df, bd_df

# --- 전체 텍스트 점수 분포 계산 (미매칭 EBD만 대상으로) ---
def compute_text_score_distribution():
    # 데이터 로드
    rule_result_df, bd_df = load_data()
    
    # RULE 매칭 결과에서 MATCH_STAGE가 "미매칭"인 건만 필터링
    unmatched_ebd = rule_result_df[rule_result_df['MATCH_STAGE'] == '미매칭']
    print("미매칭 EBD 건수:", len(unmatched_ebd))
    
    # 전처리: 미매칭 EBD와 BD 데이터에 대해 토큰 열 추가
    ebd_processed = preprocess_data(unmatched_ebd)
    bd_processed = preprocess_data(bd_df)
    
    # 전체 EBD의 BD 후보 점수 계산: 각 EBD 행의 RECAP_PK에 해당하는 BD 후보들과 텍스트 점수 계산
    results = []
    for ebd_idx, ebd_row in tqdm(ebd_processed.iterrows(), total=len(ebd_processed), desc="EBD 처리"):
        recap = ebd_row['RECAP_PK']
        # 해당 RECAP_PK와 일치하는 BD 후보들 선택
        bd_candidates = bd_processed[bd_processed['RECAP_PK'] == recap]
        if bd_candidates.empty:
            continue
        for bd_idx, bd_row in bd_candidates.iterrows():
            office, bld, juso, total_text = calculate_text_score(ebd_row, bd_row)
            results.append({
                'EBD_idx': ebd_idx,
                'BD_idx': bd_idx,
                'RECAP_PK': recap,
                'office_score': office,
                'bld_score': bld,
                'juso_score': juso,
                'total_text_score': total_text
            })
    
    all_scores_df = pd.DataFrame(results)
    
    # 전체 점수 분포 집계
    print("\n전체 텍스트 총점 분포:")
    print(all_scores_df['total_text_score'].value_counts())
    
    print("\nOffice score 분포:")
    print(all_scores_df['office_score'].value_counts())
    
    print("\nBld score 분포:")
    print(all_scores_df['bld_score'].value_counts())
    
    print("\nJuso score 분포:")
    print(all_scores_df['juso_score'].value_counts())
    
    # 예시: 전체 텍스트 점수가 3점인 경우 (모든 항목에서 일치한 경우)
    three_point_cases = all_scores_df[all_scores_df['total_text_score'] == 3.0]
    print("\n총 텍스트 점수 3점인 후보 수:", len(three_point_cases))
    
    return all_scores_df

if __name__ == "__main__":    
    scores_df = compute_text_score_distribution()


미매칭 EBD 건수: 1726


EBD 처리: 100%|██████████| 1726/1726 [00:04<00:00, 425.24it/s]



전체 텍스트 총점 분포:
total_text_score
0.0    20171
1.0     6138
2.0     2974
3.0      217
Name: count, dtype: int64

Office score 분포:
office_score
0.0    24087
1.0     5413
Name: count, dtype: int64

Bld score 분포:
bld_score
0.0    26837
1.0     2663
Name: count, dtype: int64

Juso score 분포:
juso_score
0.0    24839
1.0     4661
Name: count, dtype: int64

총 텍스트 점수 3점인 후보 수: 217


In [17]:
# %% [code]
import os
import re
import pandas as pd
from tqdm import tqdm

# --- 전처리 함수: 텍스트 토큰화 ---
def tokenize_text(text):
    """
    소문자 변환, 특수문자(괄호 등)를 모두 공백으로 치환 후,
    공백 기준으로 토큰 분할하여 리스트를 반환합니다.
    """
    if pd.isna(text):
        return []
    text = text.lower()
    # 영문, 숫자, 한글, 공백 외의 모든 문자 -> 공백으로 치환
    text = re.sub(r'[^a-z0-9가-힣\s]', ' ', text)
    text = ' '.join(text.split())
    return text.split()

# --- 데이터 전처리 함수 ---
def preprocess_data(df, is_bd=False):
    """
    DataFrame 내 텍스트 컬럼에 대해 토큰화 열을 추가합니다.
    - EBD의 경우: '기관명', '건축물명', '주소', 'RECAP_PK'
    - BD의 경우: 'BLD_NM', 'DONG_NM', 'RECAP_PK'
    """
    df = df.copy()
    if is_bd:
        cols_to_tokenize = ['BLD_NM', 'DONG_NM', 'RECAP_PK']
    else:
        cols_to_tokenize = ['기관명', '건축물명', '주소', 'RECAP_PK']
    for col in cols_to_tokenize:
        if col in df.columns:
            df[col + '_tokens'] = df[col].apply(tokenize_text)
    return df

# --- 텍스트 점수 계산 함수 (버전 2: 통합 토큰 비교) ---
def calculate_text_score_v2(ebd_row, bd_row):
    """
    EBD의 3개 텍스트 변수(기관명, 건축물명, 주소)에서 나온 토큰들을 통합(중복 제거)한 집합과,
    BD의 2개 텍스트 변수(BLD_NM, DONG_NM)에서 나온 토큰들을 통합한 집합을 비교합니다.
    - BLD_NM 토큰 집합과 EBD 통합 토큰 집합에 1개 이상 일치하면 bld_score = 1, 아니면 0.
    - DONG_NM 토큰 집합과 EBD 통합 토큰 집합에 1개 이상 일치하면 dong_score = 1, 아니면 0.
    최종 total_text_score는 bld_score + dong_score (0, 1, 또는 2).
    """
    # EBD 통합 토큰 집합 (중복 제거)
    ebd_unified = set(ebd_row.get('기관명_tokens', [])) | set(ebd_row.get('건축물명_tokens', [])) | set(ebd_row.get('주소_tokens', []))
    # BD 통합 토큰 집합
    bd_unified = set(bd_row.get('BLD_NM_tokens', [])) | set(bd_row.get('DONG_NM_tokens', []))
    
    # 점수 부여: 각각 1점씩 (토큰이 1개 이상 겹치면)
    bld_score = 1.0 if ebd_unified.intersection(set(bd_row.get('BLD_NM_tokens', []))) else 0.0
    dong_score = 1.0 if ebd_unified.intersection(set(bd_row.get('DONG_NM_tokens', []))) else 0.0
    total_text_score = bld_score + dong_score
    return bld_score, dong_score, total_text_score

# --- 데이터 로드 함수 ---
def load_data():
    """
    RULE 매칭 결과(EBD 데이터)와 BD_all 데이터를 엑셀 파일에서 로드합니다.
    파일 경로는 실제 환경에 맞게 수정하세요.
    """
    rule_result_df = pd.read_excel("../result/rule_matching_result_ver3.xlsx")
    bd_df = pd.read_excel("../data/BD_data_all.xlsx")
    return rule_result_df, bd_df

# --- 전체 텍스트 점수 및 토큰 정보 통합 함수 ---
def compute_text_scores_with_tokens():
    """
    RULE 매칭 결과에서 MATCH_STAGE가 '미매칭'인 EBD 데이터에 대해,
    각 EBD 행의 RECAP_PK와 일치하는 BD 후보들을 대상으로
    EBD의 통합 토큰과 BD의 통합 토큰, 그리고 텍스트 점수를 계산하고,
    그 결과를 DataFrame으로 반환합니다.
    """
    # 데이터 로드
    rule_result_df, bd_df = load_data()
    
    # 미매칭 EBD 데이터만 선택
    unmatched_ebd = rule_result_df[rule_result_df['MATCH_STAGE'] == '미매칭']
    print("미매칭 EBD 건수:", len(unmatched_ebd))
    
    # 전처리: 미매칭 EBD와 BD 데이터 토큰화
    ebd_processed = preprocess_data(unmatched_ebd, is_bd=False)
    bd_processed = preprocess_data(bd_df, is_bd=True)
    
    results = []
    # 각 EBD 행에 대해 처리 (RECAP_PK 기준으로 BD 후보 선택)
    for ebd_idx, ebd_row in tqdm(ebd_processed.iterrows(), total=len(ebd_processed), desc="EBD 처리"):
        recap = ebd_row['RECAP_PK']
        bd_candidates = bd_processed[bd_processed['RECAP_PK'] == recap]
        if bd_candidates.empty:
            continue
        # EBD 통합 토큰 (중복 제거 후 정렬)
        ebd_unified_tokens = sorted(list(set(ebd_row.get('기관명_tokens', [])) |
                                           set(ebd_row.get('건축물명_tokens', [])) |
                                           set(ebd_row.get('주소_tokens', []))))
        for bd_idx, bd_row in bd_candidates.iterrows():
            # BD 통합 토큰
            bd_unified_tokens = sorted(list(set(bd_row.get('BLD_NM_tokens', [])) |
                                            set(bd_row.get('DONG_NM_tokens', []))))
            bld_score, dong_score, total_text_score = calculate_text_score_v2(ebd_row, bd_row)
            results.append({
                'EBD_idx': ebd_idx,
                'BD_idx': bd_idx,
                'RECAP_PK': recap,
                'ebd_unified_tokens': ebd_unified_tokens,
                'bd_unified_tokens': bd_unified_tokens,
                'bld_score': bld_score,
                'dong_score': dong_score,
                'total_text_score': total_text_score
            })
    return pd.DataFrame(results)

# --- 실행: 전체 결과 계산 및 분포 출력 ---
df_scores = compute_text_scores_with_tokens()
print("미매칭 EBD의 BD 후보별 텍스트 점수 및 토큰 정보:")
display(df_scores.head())

print("\n전체 텍스트 총점 분포:")
print(df_scores['total_text_score'].value_counts())

print("\nBLD score 분포:")
print(df_scores['bld_score'].value_counts())

print("\nDONG score 분포:")
print(df_scores['dong_score'].value_counts())


미매칭 EBD 건수: 1726


EBD 처리: 100%|██████████| 1726/1726 [00:09<00:00, 189.89it/s]


미매칭 EBD의 BD 후보별 텍스트 점수 및 토큰 정보:


,EBD_idx,BD_idx,RECAP_PK,ebd_unified_tokens,bd_unified_tokens,bld_score,dong_score,total_text_score
0,2,286,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[본관],0.0,0.0,0.0
1,2,287,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제4별관],0.0,0.0,0.0
2,2,288,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제1별관],0.0,0.0,0.0
3,2,289,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제5호],0.0,0.0,0.0
4,2,290,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제4호],0.0,0.0,0.0



전체 텍스트 총점 분포:
total_text_score
0.0    20171
1.0     8851
2.0      478
Name: count, dtype: int64

BLD score 분포:
bld_score
0.0    20791
1.0     8709
Name: count, dtype: int64

DONG score 분포:
dong_score
0.0    28402
1.0     1098
Name: count, dtype: int64


In [18]:
df_scores

,EBD_idx,BD_idx,RECAP_PK,ebd_unified_tokens,bd_unified_tokens,bld_score,dong_score,total_text_score
0,2,286,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[본관],0.0,0.0,0.0
1,2,287,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제4별관],0.0,0.0,0.0
2,2,288,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제1별관],0.0,0.0,0.0
3,2,289,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제5호],0.0,0.0,0.0
4,2,290,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제4호],0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
29495,4189,5557,36110-1000000000000000571160,"[187번지, 2동, 3, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",1.0,0.0,1.0
29496,4190,5557,36110-1000000000000000571160,"[187번지, 3, 3동, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",1.0,0.0,1.0
29497,4191,5557,36110-1000000000000000571160,"[187번지, 3, 4동, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",1.0,0.0,1.0
29498,4192,5557,36110-1000000000000000571160,"[187번지, 3, 5동, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",1.0,0.0,1.0


In [19]:
df_scores[df_scores['total_text_score']==2]

,EBD_idx,BD_idx,RECAP_PK,ebd_unified_tokens,bd_unified_tokens,bld_score,dong_score,total_text_score
162,109,12438,51150-100303040,"[1113, 강릉경찰서, 강릉시, 강원특별자치도, 강원특별자치도경찰청, 포남동]","[강릉경찰서, 직장어린이집]",1.0,1.0,2.0
215,134,7125,41590-6878,"[14, 315, 경기도, 경기도농업기술원, 기산동, 본관, 화성시]","[1동, 경기도농업기술원, 본관]",1.0,1.0,2.0
275,148,6085,41192-336,"[1218, 경기도, 부천소방서, 부천시, 원미구, 중동, 청사동]","[부천소방서, 청사동]",1.0,1.0,2.0
352,183,6568,41290-100179972,"[1, 1동, 3, 경기도, 과천시, 시청사, 중앙동]","[1, 1동, 3, 과천시청, 업무시설, 중앙동]",1.0,1.0,2.0
363,184,6570,41290-100179972,"[1, 3, 3동, 경기도, 과천시, 의회청사, 중앙동]","[1, 3, 3동, 과천시청, 업무시설, 중앙동]",1.0,1.0,2.0
...,...,...,...,...,...,...,...,...
29395,4181,4458,29170-329,"[1, 1번지, 55, 광주광역시, 광주교육대학교, 국제문화예술교육센터, 북구, 풍...","[1, 광주교육대학교, 본관]",1.0,1.0,2.0
29415,4181,4478,29170-329,"[1, 1번지, 55, 광주광역시, 광주교육대학교, 국제문화예술교육센터, 북구, 풍...","[국제문화예술교육센터, 풍향문화관]",1.0,1.0,2.0
29433,4183,2906,27200-1000000000000001483139,"[1, 1797, 5번지, 남구, 대구광역시, 대구교육대학교, 대명동, 대학본부]","[대구교육대학교, 대학본부]",1.0,1.0,2.0
29464,4184,2913,27200-1000000000000001483139,"[15, 1797, 219, 5번지, 교사교육센터, 남구, 대구광역시, 대구교육대학...","[대구교육대학교, 상록교육관]",1.0,1.0,2.0


In [20]:
import os
import re
import pandas as pd
from tqdm import tqdm

# --- 전처리 함수: 텍스트 토큰화 ---
def tokenize_text(text):
    """
    소문자 변환, 특수문자(괄호 등)를 모두 공백으로 치환 후,
    공백 기준으로 토큰 분할하여 리스트를 반환합니다.
    """
    if pd.isna(text):
        return []
    text = text.lower()
    # 영문, 숫자, 한글, 공백 외의 모든 문자 -> 공백으로 치환
    text = re.sub(r'[^a-z0-9가-힣\s]', ' ', text)
    text = ' '.join(text.split())
    return text.split()

# --- 데이터 전처리 함수 ---
def preprocess_data(df, is_bd=False):
    """
    DataFrame 내 텍스트 컬럼에 대해 토큰화 열을 추가합니다.
    - EBD의 경우: '기관명', '건축물명', '주소', 'RECAP_PK'
    - BD의 경우: 'BLD_NM', 'DONG_NM', 'RECAP_PK'
    """
    df = df.copy()
    if is_bd:
        cols_to_tokenize = ['BLD_NM', 'DONG_NM', 'RECAP_PK']
    else:
        cols_to_tokenize = ['기관명', '건축물명', '주소', 'RECAP_PK']
    for col in cols_to_tokenize:
        if col in df.columns:
            df[col + '_tokens'] = df[col].apply(tokenize_text)
    return df

# --- 텍스트 점수 계산 함수 (버전 2: 분리된 BD 토큰 비교) ---
def calculate_text_score_v2(ebd_row, bd_row):
    """
    EBD의 3개 텍스트 변수(기관명, 건축물명, 주소)에서 나온 토큰들을 통합(중복 제거)한 집합과,
    BD의 2개 텍스트 변수 토큰(별도 저장된 BLD_NM_tokens, DONG_NM_tokens)을 비교합니다.
    
    각각:
      - BLD_NM_tokens와 통합 토큰 집합의 교집합이 있으면 bld_score = 1, 없으면 0.
      - DONG_NM_tokens와 통합 토큰 집합의 교집합이 있으면 dong_score = 1, 없으면 0.
      
    최종 total_text_score는 bld_score + dong_score (0, 1, 또는 2).
    """
    # EBD 통합 토큰 집합 (중복 제거)
    ebd_unified = set(ebd_row.get('기관명_tokens', [])) | set(ebd_row.get('건축물명_tokens', [])) | set(ebd_row.get('주소_tokens', []))
    
    # BD의 각 토큰은 분리된 열로 유지됨.
    bld_tokens = set(bd_row.get('BLD_NM_tokens', []))
    dong_tokens = set(bd_row.get('DONG_NM_tokens', []))
    
    bld_score = 1.0 if ebd_unified.intersection(bld_tokens) else 0.0
    dong_score = 1.0 if ebd_unified.intersection(dong_tokens) else 0.0
    total_text_score = bld_score + dong_score
    return bld_score, dong_score, total_text_score

# --- 데이터 로드 함수 ---
def load_data():
    """
    RULE 매칭 결과(EBD 데이터)와 BD_all 데이터를 엑셀 파일에서 로드합니다.
    파일 경로는 실제 환경에 맞게 수정하세요.
    """
    rule_result_df = pd.read_excel("../result/rule_matching_result_ver3.xlsx")
    bd_df = pd.read_excel("../data/BD_data_all.xlsx")
    return rule_result_df, bd_df

# --- 전체 텍스트 점수 및 토큰 정보 통합 함수 ---
def compute_text_scores_with_tokens():
    """
    RULE 매칭 결과에서 MATCH_STAGE가 '미매칭'인 EBD 데이터에 대해,
    각 EBD 행의 RECAP_PK와 일치하는 BD 후보들을 대상으로
    EBD의 통합 토큰(중복 제거)과 BD의 분리된 토큰(BLD_NM_tokens, DONG_NM_tokens) 및
    텍스트 점수를 계산하여, 그 결과를 DataFrame으로 반환합니다.
    """
    # 데이터 로드
    rule_result_df, bd_df = load_data()
    
    # 미매칭 EBD 데이터만 선택
    unmatched_ebd = rule_result_df[rule_result_df['MATCH_STAGE'] == '미매칭']
    print("미매칭 EBD 건수:", len(unmatched_ebd))
    
    # 전처리: 미매칭 EBD와 BD 데이터 토큰화
    ebd_processed = preprocess_data(unmatched_ebd, is_bd=False)
    bd_processed = preprocess_data(bd_df, is_bd=True)
    
    results = []
    # 각 EBD 행에 대해 처리 (RECAP_PK 기준으로 BD 후보 선택)
    for ebd_idx, ebd_row in tqdm(ebd_processed.iterrows(), total=len(ebd_processed), desc="EBD 처리"):
        recap = ebd_row['RECAP_PK']
        bd_candidates = bd_processed[bd_processed['RECAP_PK'] == recap]
        if bd_candidates.empty:
            continue
        # EBD 통합 토큰 (중복 제거 후 정렬하여 저장)
        ebd_unified_tokens = sorted(list(set(ebd_row.get('기관명_tokens', [])) |
                                           set(ebd_row.get('건축물명_tokens', [])) |
                                           set(ebd_row.get('주소_tokens', []))))
        for bd_idx, bd_row in bd_candidates.iterrows():
            # BD 토큰은 별도로 저장 (합치지 않고, 분리된 상태)
            bld_tokens = sorted(list(set(bd_row.get('BLD_NM_tokens', []))))
            dong_tokens = sorted(list(set(bd_row.get('DONG_NM_tokens', []))))
            
            bld_score, dong_score, total_text_score = calculate_text_score_v2(ebd_row, bd_row)
            results.append({
                'EBD_idx': ebd_idx,
                'BD_idx': bd_idx,
                'RECAP_PK': recap,
                'ebd_unified_tokens': ebd_unified_tokens,
                'bld_tokens': bld_tokens,
                'dong_tokens': dong_tokens,
                'bld_score': bld_score,
                'dong_score': dong_score,
                'total_text_score': total_text_score
            })
    return pd.DataFrame(results)

# --- 실행: 전체 결과 계산 및 분포 출력 ---
df_scores = compute_text_scores_with_tokens()
print("미매칭 EBD의 BD 후보별 텍스트 점수 및 토큰 정보:")
display(df_scores.head())

print("\n전체 텍스트 총점 분포:")
print(df_scores['total_text_score'].value_counts())

print("\nBLD score 분포:")
print(df_scores['bld_score'].value_counts())

print("\nDONG score 분포:")
print(df_scores['dong_score'].value_counts())


미매칭 EBD 건수: 1726


EBD 처리: 100%|██████████| 1726/1726 [00:04<00:00, 363.20it/s]

미매칭 EBD의 BD 후보별 텍스트 점수 및 토큰 정보:


,EBD_idx,BD_idx,RECAP_PK,ebd_unified_tokens,bld_tokens,dong_tokens,bld_score,dong_score,total_text_score
0,2,286,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[본관],[],0.0,0.0,0.0
1,2,287,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제4별관],[],0.0,0.0,0.0
2,2,288,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제1별관],[],0.0,0.0,0.0
3,2,289,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제5호],[],0.0,0.0,0.0
4,2,290,11110-900,"[102, 28, 감사원, 삼청동, 서울특별시, 제2별관, 제3별관, 종로구]",[제4호],[],0.0,0.0,0.0



전체 텍스트 총점 분포:
total_text_score
0.0    20171
1.0     8851
2.0      478
Name: count, dtype: int64

BLD score 분포:
bld_score
0.0    20791
1.0     8709
Name: count, dtype: int64

DONG score 분포:
dong_score
0.0    28402
1.0     1098
Name: count, dtype: int64


In [28]:
all_scores=df_scores[df_scores['total_text_score']==2]
all_scores

,EBD_idx,BD_idx,RECAP_PK,ebd_unified_tokens,bld_tokens,dong_tokens,bld_score,dong_score,total_text_score
162,109,12438,51150-100303040,"[1113, 강릉경찰서, 강릉시, 강원특별자치도, 강원특별자치도경찰청, 포남동]","[강릉경찰서, 직장어린이집]","[강릉경찰서, 직장어린이집]",1.0,1.0,2.0
215,134,7125,41590-6878,"[14, 315, 경기도, 경기도농업기술원, 기산동, 본관, 화성시]",[경기도농업기술원],"[1동, 본관]",1.0,1.0,2.0
275,148,6085,41192-336,"[1218, 경기도, 부천소방서, 부천시, 원미구, 중동, 청사동]",[부천소방서],[청사동],1.0,1.0,2.0
352,183,6568,41290-100179972,"[1, 1동, 3, 경기도, 과천시, 시청사, 중앙동]","[1, 3, 과천시청, 업무시설, 중앙동]",[1동],1.0,1.0,2.0
363,184,6570,41290-100179972,"[1, 3, 3동, 경기도, 과천시, 의회청사, 중앙동]","[1, 3, 과천시청, 업무시설, 중앙동]",[3동],1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...
29395,4181,4458,29170-329,"[1, 1번지, 55, 광주광역시, 광주교육대학교, 국제문화예술교육센터, 북구, 풍...",[광주교육대학교],"[1, 본관]",1.0,1.0,2.0
29415,4181,4478,29170-329,"[1, 1번지, 55, 광주광역시, 광주교육대학교, 국제문화예술교육센터, 북구, 풍...",[풍향문화관],[국제문화예술교육센터],1.0,1.0,2.0
29433,4183,2906,27200-1000000000000001483139,"[1, 1797, 5번지, 남구, 대구광역시, 대구교육대학교, 대명동, 대학본부]",[대구교육대학교],[대학본부],1.0,1.0,2.0
29464,4184,2913,27200-1000000000000001483139,"[15, 1797, 219, 5번지, 교사교육센터, 남구, 대구광역시, 대구교육대학...",[대구교육대학교],[상록교육관],1.0,1.0,2.0


In [25]:
df_scores[df_scores['dong_score']==1]

,EBD_idx,BD_idx,RECAP_PK,ebd_unified_tokens,bld_tokens,dong_tokens,bld_score,dong_score,total_text_score
28,9,4864,30200-100183053,"[2번지, 32, 481, 과학기술전시체험센터, 과학기술정보통신부, 구성동, 국립중...",[],[과학기술전시체험센터],0.0,1.0,1.0
95,73,12477,51150-1185,"[1001, 강릉시, 강릉시청사, 강원특별자치도, 홍제동]",[],[강릉시청사],0.0,1.0,1.0
118,74,12482,51150-1185,"[1001, 강릉시, 강릉시의회청사, 강원특별자치도, 홍제동]",[],[강릉시의회청사],0.0,1.0,1.0
140,84,12307,51130-100205802,"[909, a동, 강원특별자치도, 단계동, 원주시, 청소년수련관]",[],"[a동, b동, c동]",0.0,1.0,1.0
144,85,12307,51130-100205802,"[909, b동, 강원특별자치도, 원주시, 청소년수련관]",[],"[a동, b동, c동]",0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
29395,4181,4458,29170-329,"[1, 1번지, 55, 광주광역시, 광주교육대학교, 국제문화예술교육센터, 북구, 풍...",[광주교육대학교],"[1, 본관]",1.0,1.0,2.0
29415,4181,4478,29170-329,"[1, 1번지, 55, 광주광역시, 광주교육대학교, 국제문화예술교육센터, 북구, 풍...",[풍향문화관],[국제문화예술교육센터],1.0,1.0,2.0
29433,4183,2906,27200-1000000000000001483139,"[1, 1797, 5번지, 남구, 대구광역시, 대구교육대학교, 대명동, 대학본부]",[대구교육대학교],[대학본부],1.0,1.0,2.0
29464,4184,2913,27200-1000000000000001483139,"[15, 1797, 219, 5번지, 교사교육센터, 남구, 대구광역시, 대구교육대학...",[대구교육대학교],[상록교육관],1.0,1.0,2.0


In [27]:
df_scores[df_scores['bld_score']==1]

,EBD_idx,BD_idx,RECAP_PK,ebd_unified_tokens,bld_tokens,dong_tokens,bld_score,dong_score,total_text_score
25,9,4861,30200-100183053,"[2번지, 32, 481, 과학기술전시체험센터, 과학기술정보통신부, 구성동, 국립중...",[국립중앙과학관],[],1.0,0.0,1.0
27,9,4863,30200-100183053,"[2번지, 32, 481, 과학기술전시체험센터, 과학기술정보통신부, 구성동, 국립중...",[국립중앙과학관],[천체관측소],1.0,0.0,1.0
37,10,4861,30200-100183053,"[2, 32, 과학기술정보통신부, 구성동, 국립중앙과학관, 대전광역시, 유성구, 특...",[국립중앙과학관],[],1.0,0.0,1.0
39,10,4863,30200-100183053,"[2, 32, 과학기술정보통신부, 구성동, 국립중앙과학관, 대전광역시, 유성구, 특...",[국립중앙과학관],[천체관측소],1.0,0.0,1.0
49,11,4861,30200-100183053,"[2, 32, 과학기술정보통신부, 관리동, 구성동, 국립중앙과학관, 대전광역시, 유성구]",[국립중앙과학관],[],1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
29495,4189,5557,36110-1000000000000000571160,"[187번지, 2동, 3, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",[],1.0,0.0,1.0
29496,4190,5557,36110-1000000000000000571160,"[187번지, 3, 3동, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",[],1.0,0.0,1.0
29497,4191,5557,36110-1000000000000000571160,"[187번지, 3, 4동, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",[],1.0,0.0,1.0
29498,4192,5557,36110-1000000000000000571160,"[187번지, 3, 5동, 세종청사, 세종특별자치도, 세종특별자치시, 어진동, 정부...","[정부세종청사, 중앙동]",[],1.0,0.0,1.0
